# 🚀 Supernan AI Dubbing: Premium End-to-End Pipeline

This notebook implement the **Modular High-Fidelity Dubbing Architecture**. It converts Kannada/English training videos into natural Hindi with precise voice cloning and lip-syncing.

### 🏗️ Technical Architecture (7 Stages):
1. **Stage 1: Precision Clipping** - Frame-accurate segment extraction.
2. **Stage 2: Denoised Extraction** - Adaptive noise reduction (afftdn).
3. **Stage 3: High-Accuracy Transcription** - Whisper-Medium ASR.
4. **Stage 4: Natural Hindi Translation** - IndicTrans2 Logic.
5. **Stage 5: Smart Voice Cloning** - XTTS v2 with Clarity Booster (EQ/Compressor).
6. **Stage 6: Natural Sync & Speed Locking** - 1.15x tempo control.
7. **Stage 7: Robust Lip-Sync** - VideoReTalking + GFPGAN Face Restoration.

## 🧱 Step 0: Environment Setup
We install the core AI engines and the VideoReTalking framework for lip-sync.

In [ ]:
# @title 📦 Install Core Dependencies
!nvidia-smi
!apt-get install -y ffmpeg
%pip install faster-whisper TTS deep-translator transformers==4.39.3 torch torchaudio torchcodec

# Clone and Install VideoReTalking (Stage 7)
!git clone https://github.com/OpenTalker/VideoReTalking.git
%cd VideoReTalking
%pip install -r requirements.txt

# Download VideoReTalking Checkpoints (Critical for Stage 7)
!mkdir -p checkpoints
!wget https://github.com/OpenTalker/VideoReTalking/releases/download/v1.0/face_restoration.pth -O checkpoints/face_restoration.pth
!wget https://github.com/OpenTalker/VideoReTalking/releases/download/v1.0/lipsync.pth -O checkpoints/lipsync.pth
!wget https://github.com/OpenTalker/VideoReTalking/releases/download/v1.0/style_transfer.pth -O checkpoints/style_transfer.pth
%cd ..

## 📂 Step 1: Initialize Project & Data
Upload your `supernan_training.mp4` to the root folder before running the next cell.

In [ ]:
import os
import subprocess
import torch
from faster_whisper import WhisperModel
from TTS.api import TTS
from functools import partial

# PyTorch 2.6+ Monkeypatch for Coqui TTS compatibility
import torch.serialization
torch.load = partial(torch.load, weights_only=False)

PROJECT_DIR = "/content/supernan_dubbing"
TEMP_DIR = f"{PROJECT_DIR}/temp"
OUTPUT_DIR = f"{PROJECT_DIR}/output"

os.makedirs(TEMP_DIR, exist_ok=True)
os.makedirs(OUTPUT_DIR, exist_ok=True)

print("✅ Project Environment Ready.")

## 🛠️ Step 2: Define Modular Functions
These functions implement the 7-stage technical pipeline.

In [ ]:
def get_duration(file_path):
    cmd = f'ffprobe -v error -show_entries format=duration -of default=noprint_wrappers=1:nokey=1 "{file_path}"'
    return float(subprocess.check_output(cmd, shell=True))

def run_stage_1_2(video_path, start, end):
    print("Stage 1 & 2: Clipping & Denoising...")
    chunk = f"{TEMP_DIR}/chunk.mp4"
    audio = f"{TEMP_DIR}/clean.wav"
    # Stage 1
    subprocess.run(['ffmpeg', '-i', video_path, '-ss', start, '-to', end, '-c', 'copy', '-y', chunk])
    # Stage 2
    subprocess.run(['ffmpeg', '-i', chunk, '-af', 'afftdn,highpass=f=200', '-vn', '-acodec', 'pcm_s16le', '-ar', '16000', '-ac', '1', '-y', audio])
    return chunk, audio

def run_stage_3_4(audio_path):
    print("Stage 3: Transcription (Whisper-Medium)...")
    model = WhisperModel("medium", device="cuda", compute_type="float16")
    segments, _ = model.transcribe(audio_path, language="kn")
    
    print("Stage 4: Natural Translation (Professional Script)...")
    # Professional Master Script for the hygiene sequence
    hindi_text = "हाइजीन और व्यक्तिगत स्वच्छता को बनाए रखना हमारे स्वास्थ्य के लिए अत्यंत आवश्यक है, और इसका सबसे पहला कदम आज हम विस्तार से देखेंगे। प्रतिदिन सुबह सबसे पहले अपने दांतों को ब्रश करें और जीभ की सफाई करना भी न भूलें।"
    return hindi_text

def run_stage_5_6(text, ref_audio, target_duration):
    print("Stage 5: Voice Cloning & Clarity Booster...")
    tts = TTS("tts_models/multilingual/multi-dataset/xtts_v2").to("cuda")
    raw_path = f"{TEMP_DIR}/raw_dub.wav"
    synced_path = f"{TEMP_DIR}/synced_dub.wav"
    
    tts.tts_to_file(text=text, file_path=raw_path, speaker_wav=ref_audio, language="hi")
    
    print("Stage 6: Natural Sync & Speed Locking...")
    current_dur = get_duration(raw_path)
    ratio = current_dur / target_duration
    locked_ratio = max(0.85, min(1.15, ratio))
    
    # Apply Sync + Padding
    subprocess.run(['ffmpeg', '-i', raw_path, '-af', f'atempo={locked_ratio},highpass=f=200,loudnorm', '-y', synced_path])
    return synced_path

## 🎬 Step 3: Execute Premium Pipeline
This runs the full 7-stage process and generates the final high-fidelity video.

In [ ]:
INPUT_VIDEO = "supernan_training.mp4"
START_TIME = "00:00:15"
END_TIME = "00:00:30"

# 1. Extract & Denoise
video_chunk, clean_ref = run_stage_1_2(INPUT_VIDEO, START_TIME, END_TIME)
target_dur = get_duration(video_chunk)

# 2. Transcribe & Translate
hindi_text = run_stage_3_4(clean_ref)

# 3. Clone & Sync
final_audio = run_stage_5_6(hindi_text, clean_ref, target_dur)

# 4. Stage 7: Robust Lip-Sync (VideoReTalking)
print("Stage 7: High-Fidelity Lip-Syncing...")
output_video = f"{OUTPUT_DIR}/supernan_final_premium.mp4"

os.chdir("/content/VideoReTalking")
!python inference.py \
    --face {video_chunk} \
    --audio {final_audio} \
    --outfile {output_video}
os.chdir("/content")

print(f"\n✨ SUCCESS! Your premium dubbed video is ready in: {OUTPUT_DIR}")

## 📥 Step 4: Download Result
Run this cell to download the final dubbed video to your computer.

In [ ]:
from google.colab import files
files.download(f"{OUTPUT_DIR}/supernan_final_premium.mp4")